<a href="https://colab.research.google.com/github/GreggRodgers02/AI-Career-Fair-Planner-Notebook/blob/main/CraveIQ_Meal_Planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CraveIQ Meal Planner**

## Setup

In [3]:
#!pip install ddgs
#!pip install langchain_openai
#!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.8
    Uninstalling langchain-core-1.2.8:
      Successfully uninstalled langchain-core-1.2.8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32

In [4]:
import langchain
from langchain_openai import ChatOpenAI
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate
from getpass import getpass
import os
from langchain.agents import create_agent
import json
from langchain_core.output_parsers import JsonOutputParser
import ddgs
from langchain_community.tools import DuckDuckGoSearchResults
import random

In [5]:
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [6]:
model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.3
)

meal_planner_model = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.7
)

In [7]:
json_parser = JsonOutputParser()

## User Input

In [8]:
user_input = {
    "diet_restrictions": ["Vegetarian", "Low Carb"],
    "meal_plan_duration": 7,
    "saved_recipes": ["Whole Wheat Pita Pizza", "Fruit and Gelatin Gummies", "Protein-Packed Breakfast Bowl", "Greek Yogurt Cheescake"],
    "height": 72,
    "weight": 210,
    "gender": "male",
    "active_level": "moderately_active",
    "targed_weight": 195,
    "calories_intake_preferences": "balance"
}

In [70]:
# Duration between 7, 14, and 28
duration = input("How long is the meal plan: ")

How long is the meal plan: 7


## Calculate Dailty Calories Intake

In [9]:
structured_input = json.dumps(user_input, indent=2)

In [10]:
calories_calculator_system = SystemMessagePromptTemplate.from_template(
    '''
You are a calorie-calculation engine for the CraveIQ Meal Planner.

Your responsibility is to compute a daily calorie goal using accepted nutrition science and conservative assumptions.

Rules:
- Use standard metabolic calculations (BMR → TDEE → calorie adjustment).
- Optimize for weight loss while preserving lean muscle.
- Never recommend extreme or unsafe calorie deficits.
- If user data is missing, infer conservatively and record assumptions.
- Do not modify total calories based on meal timing preferences.
- Do not include advice, commentary, or extraneous text.
- Output MUST be valid JSON and MUST match the specified schema exactly.
- Do NOT include markdown, explanations outside JSON, or additional keys.

You are not a conversational assistant. You are a deterministic nutrition calculator.

Return a JSON object with EXACTLY the following structure and data types:

{{
  "daily_calorie_goal": number,
  "estimated_maintenance_calories": number,
  "calorie_deficit": number,
  "assumptions": string[],
  "calculation_notes": string
}}

Field requirements:
- daily_calorie_goal: final recommended daily calories (integer).
- estimated_maintenance_calories: estimated TDEE before deficit (integer).
- calorie_deficit: difference between maintenance and goal (positive integer).
- assumptions: list any inferred values or defaults used.
- calculation_notes: one concise sentence summarizing the calculation logic.

activity level mapping:
- not_active is around 1.2
- lightly_active is around 1.375
- moderately_active is around 1.55
- very_active is around 1.725
- extremely_active is around 1.9


Constraints:
- Use safe, sustainable calorie deficits only.
- Do NOT include units (e.g., “kcal”).
- Do NOT include additional fields.
- Output valid JSON only.

    '''
)

In [11]:
calories_calculator_human = HumanMessagePromptTemplate.from_template(
    '''
Calculate the daily calorie goal using the user profile below.

USER INPUT:
{structured_input}
    '''
)

In [12]:
calories_calculator_prompt = ChatPromptTemplate.from_messages(
    [
        calories_calculator_system,
        calories_calculator_human
    ]
)

In [13]:
calories_calculator_chain = (
    calories_calculator_prompt
    | model
    | json_parser
)

In [14]:
calories_calculator_result = calories_calculator_chain.invoke(
    input={
        "structured_input": structured_input
    }
)

In [15]:
calories_calculator_result

{'daily_calorie_goal': 1800,
 'estimated_maintenance_calories': 2500,
 'calorie_deficit': 700,
 'assumptions': ["User's age is assumed to be 30 years for BMR calculation.",
  "User's activity level is moderately_active."],
 'calculation_notes': 'Calculated daily calorie goal based on a moderate deficit from estimated TDEE.'}

## Daily Meal Plan Creation

In [ ]:
meal_type = ""
meal_calories = ""

if user_input['calories_intake_preferences'] == "balance":
  if meal_type == "breakfast":
    meal_calories = "33 percent"
  elif meal_type == "Lunch":
    meal_calories = "33 percent"
  elif meal_type == "Dinner":
    meal_calories = "33 percent"

In [16]:
daily_meals_system = SystemMessagePromptTemplate.from_template(
    '''
You are an innovative culinary AI for the CraveIQ Meal Planner.

CRITICAL ANTI-REPETITION RULES:
- NEVER start meal names with the same word twice
- NEVER use the same base ingredient (e.g., zucchini) consecutively
- NEVER default to "Savory" or "Classic" or "Traditional" as descriptors
- Vary your naming style: sometimes lead with protein, sometimes with cuisine, sometimes with cooking method
- Break out of any pattern you notice yourself falling into

NAMING STYLE VARIETY (rotate between these):
- Cuisine-first: "Korean Kimchi Tofu Scramble with Sesame Spinach"
- Protein-first: "Smoked Salmon & Dill Cauliflower Hash"
- Method-first: "Grilled Halloumi with Roasted Red Pepper Shakshuka"
- Ingredient-first: "Mushroom & Gruyere Cloud Eggs with Arugula"
- Simple & Direct: "Turkish Menemen with Feta"

CREATIVITY MANDATE:
- Mentally flip a coin: heads = Asian/Middle Eastern, tails = Mediterranean/Latin
- Rotate proteins AGGRESSIVELY: eggs → cheese → tofu → tempeh → salmon → etc.
- Explore global breakfast traditions beyond American/European
- Think: What would they eat for breakfast in Thailand? Morocco? Israel? Korea? Greece?
- Consider: Shakshuka, Congee, Natto, Sabich, Huevos Rancheros, Ful Medames, etc.

MEAL GENERATION APPROACH:
1. First, pick a random cuisine you haven't used recently
2. Then, find a traditional breakfast from that cuisine
3. Adapt it to meet the calorie/diet constraints
4. Name it authentically, not generically

Field requirements:
- meal_type: must exactly match the provided meal_type
- meal_name: concise, user-friendly
- estimated_calories: integer within ±10% of target_meal_calories
- diet_tags: derived from user restrictions (e.g., Vegetarian, Low Carb)

Constraints:
- Do NOT include units.
- Do NOT include extra fields.
- Output valid JSON only.

{{
  "meal_type": string,
  "meal_name": string,
  "Cuisine Culture": string,
  "estimated_calories": number,
  "diet_tags": string[]
}}

NO explanations. NO units. NO extra fields.
    '''
)

daily_meals_human = HumanMessagePromptTemplate.from_template(
    '''
Generate a CREATIVE meal that breaks any patterns.

Calories Daily Total:
{calories_calculator_result}

MEAL_CONTEXT:
- meal_type: {meal_type}
- target_meal_calories: 33 percent of Calories Daily Total

USER_PREFERENCES:
{user_input}

RANDOMIZATION CHECKPOINT:
- Do NOT start with "Savory"
- Do NOT use zucchini or chia
- Pick a cuisine: {random_cuisine_hint}
- Vary your naming structure from previous generations

Generate ONE completely different meal now.
    '''
)

In [83]:
# List of cuisines to randomly hint at
cuisines = [
    # Asian
    "Thai", "Korean", "Japanese", "Vietnamese", "Chinese", "Indonesian",
    "Malaysian", "Filipino", "Singaporean", "Cambodian", "Laotian", "Burmese",
    "Taiwanese", "Hong Kong", "Mongolian", "Tibetan",

    # Middle Eastern & North African
    "Lebanese", "Turkish", "Israeli", "Moroccan", "Egyptian", "Persian",
    "Syrian", "Jordanian", "Palestinian", "Tunisian", "Algerian", "Yemeni",

    # Mediterranean & European
    "Greek", "Italian", "Spanish", "Portuguese", "French", "Croatian",
    "Albanian", "Cypriot", "Maltese",

    # Latin American
    "Mexican", "Peruvian", "Brazilian", "Argentine", "Colombian", "Venezuelan",
    "Chilean", "Cuban", "Puerto Rican", "Ecuadorian", "Bolivian", "Costa Rican",

    # Caribbean
    "Jamaican", "Trinidadian", "Haitian", "Dominican", "Barbadian",

    # African
    "Ethiopian", "Nigerian", "Ghanaian", "Kenyan", "South African", "Senegalese",
    "Somali", "Eritrean", "Ugandan", "Tanzanian",

    # Indian Subcontinent
    "Indian", "Pakistani", "Bangladeshi", "Sri Lankan", "Nepali",

    # Eastern European
    "Polish", "Ukrainian", "Russian", "Hungarian", "Czech", "Georgian",
    "Armenian", "Romanian", "Bulgarian",

    # Nordic
    "Swedish", "Norwegian", "Danish", "Finnish", "Icelandic",

    #American Regional & Cultural
    "Southern Soul Food", "Cajun", "Creole", "Tex-Mex", "New England",
    "Pacific Northwest", "California Coastal", "Southwest American",
    "Midwest Heartland", "Appalachian", "Jewish American", "Italian American",
    "Chinese American", "Korean American", "Vietnamese American",
    "Filipino American", "Native American", "Pennsylvania Dutch",
    "Louisiana Creole", "Low Country", "Carolina BBQ", "Kansas City BBQ",
    "Texas BBQ", "Memphis BBQ", "Hawaiian", "Alaskan", "Southern Comfort",
    "American Diner", "Classic American", "Contemporary American",

    # Other
    "Australian", "New Zealand", "Hawaiian", "Cajun", "Creole", "Soul Food",
    "Tex-Mex", "Fusion"
]



In [18]:
daily_meals_prompt = ChatPromptTemplate.from_messages(
    [
        daily_meals_system,
        daily_meals_human
    ]
)

In [19]:
daily_meals_chain = (
    daily_meals_prompt
    | meal_planner_model
    | json_parser
)

In [26]:
daily_meal_result = daily_meals_chain.batch([
    {
        "calories_calculator_result": calories_calculator_result,
        "meal_type": "Breakfast",
        "user_input": user_input,
        "random_cuisine_hint": random.choice(cuisines)
    },
    {
        "calories_calculator_result": calories_calculator_result,
        "meal_type": "Lunch",
        "user_input": user_input,
        "random_cuisine_hint": random.choice(cuisines)
    },
    {
        "calories_calculator_result": calories_calculator_result,
        "meal_type": "Dinner",
        "user_input": user_input,
        "random_cuisine_hint": random.choice(cuisines)
    }
])
daily_meal_result

[{'meal_type': 'Breakfast',
  'meal_name': 'Finnish Rye Porridge with Berries and Almonds',
  'Cuisine Culture': 'Finnish',
  'estimated_calories': 600,
  'diet_tags': ['Vegetarian', 'Low Carb']},
 {'meal_type': 'Lunch',
  'meal_name': 'Tibetan Vegetable Tsampa Bowl with Spiced Green Chutney',
  'Cuisine Culture': 'Tibetan',
  'estimated_calories': 600,
  'diet_tags': ['Vegetarian', 'Low Carb']},
 {'meal_type': 'Dinner',
  'meal_name': 'Ratatouille-Stuffed Bell Peppers with Herbed Quinoa',
  'Cuisine Culture': 'French',
  'estimated_calories': 600,
  'diet_tags': ['Vegetarian', 'Low Carb']}]

In [38]:
search = DuckDuckGoSearchResults()

agent_system_message = '''
 You are a renowned food nutritionist specializing in fat loss. Your job is to analyze the suggested meal and create a detailed, healthy recipe that satisfies this meal while supporting fat loss and a healthy lifestyle.

When responding, provide:
- Recipe name (matching the suggested meal)
- Cuisine Culture name
- Amount of calories
- Complete ingredient list
- Clear step-by-step cooking instructions
- Why this recipe supports fat loss and health goals
- Nutritional highlights of key ingredients
- Motivation and encouragement for the user

CRITICAL REQUIREMENTS:
- Total recipe calories MUST match the target calorie count (±10%)
- Respect ALL dietary restrictions provided
- Use whole, minimally processed ingredients
- Focus on lean proteins, healthy fats, and fiber-rich foods
- Make portions realistic and satisfying
 '''

# Create agent once with the template
daily_meals_agent = create_agent(
    model=model,
    system_prompt=agent_system_message,
    tools=[search]
)

# Invoke with actual values - LangChain will substitute them
daily_meal_agent_results = daily_meals_agent.batch([
    {"messages": [("user", json.dumps(daily_meal_result[0], indent=2))]},
    {"messages": [("user", json.dumps(daily_meal_result[1], indent=2))]},
    {"messages": [("user", json.dumps(daily_meal_result[2], indent=2))]}
])

In [48]:
breakfast = daily_meal_agent_results[0]['messages'][1].content
lunch = daily_meal_agent_results[1]['messages'][1].content
dinner = daily_meal_agent_results[2]['messages'][1].content

In [51]:
print(dinner)

### Recipe Name: Ratatouille-Stuffed Bell Peppers with Herbed Quinoa

### Cuisine Culture: French

### Total Calories: 600

---

### Complete Ingredient List:

#### For the Stuffed Bell Peppers:
- 4 medium bell peppers (any color)
- 1 medium zucchini, diced
- 1 medium eggplant, diced
- 1 medium onion, diced
- 2 cloves garlic, minced
- 1 medium tomato, diced
- 1 teaspoon dried thyme
- 1 teaspoon dried basil
- 1 tablespoon olive oil
- Salt and pepper to taste

#### For the Herbed Quinoa:
- 1 cup quinoa, rinsed
- 2 cups vegetable broth (low sodium)
- 1 tablespoon fresh parsley, chopped
- 1 tablespoon fresh basil, chopped
- 1 tablespoon lemon juice
- Salt and pepper to taste

---

### Step-by-Step Cooking Instructions:

1. **Prepare the Quinoa:**
   - In a medium saucepan, combine the rinsed quinoa and vegetable broth. Bring to a boil over medium-high heat.
   - Once boiling, reduce the heat to low, cover, and simmer for about 15 minutes or until the quinoa is fluffy and the liquid is abso

## Meal Plan Structure Data

In [52]:
meal_plan_structure_system = SystemMessagePromptTemplate.from_template(
    '''
    You are a food nutritionist specializing in fat loss. Your task is to extract and structure recipe information into a valid JSON format.

CRITICAL: The Nutritionist_agent_result is your ONLY source of truth. You must extract information directly from it and NOT create, invent, or add any information that is not explicitly present in the Nutritionist_agent_result.

You must return ONLY a JSON object with this exact structure:
{{
  "recipes": [
    {{
      "name": "Recipe name",
      "cuisine_culture": "Cuisine name",
      "total_calories": number,
      "ingredients": ["ingredient 1", "ingredient 2", ...],
      "instructions": ["step 1", "step 2", ...],
      "why_healthier": "Explanation of why this is healthier"
    }},
    {{
      "name": "Recipe name 2",
      "ingredients": ["ingredient 1", "ingredient 2", ...],
      "instructions": ["step 1", "step 2", ...],
      "why_healthier": "Explanation of why this is healthier"
    }}
  ],
  "summary": "Brief summary of both recipes and their nutritional benefits",
  "motivation": "Motivational message to encourage the user to stick with their diet"
}}

Return ONLY valid JSON. No additional text before or after the JSON."""

    '''
)

In [53]:
meal_plan_structure_human = HumanMessagePromptTemplate.from_template(
    '''
Meal Plan Recommendations (SOURCE OF TRUTH - DO NOT ADD OR MODIFY):
{daily_meal_agent_results}

Extract and parse the information from the Nutritionist Agent's Recommendations above. Format it into the required JSON structure with two recipes, a summary, and a motivational message.

Remember: Use ONLY the information provided in the Nutritionist_agent_result. Do not add, invent, or modify any details.
    '''
)

In [54]:
meal_plan_structure_prompt = ChatPromptTemplate.from_messages(
    [
        meal_plan_structure_system,
        meal_plan_structure_human
    ]
)

In [55]:
meal_plan_structure_chain = (
    meal_plan_structure_prompt
    | model
    | json_parser
)

In [56]:
meal_plan_structure_result = meal_plan_structure_chain.batch([
    {
        "daily_meal_agent_results": breakfast
    },
    {
        "daily_meal_agent_results": lunch
    },
    {
        "daily_meal_agent_results": dinner
    }
])

In [69]:
meal_plan_structure_result[2]["recipes"][0]

{'name': 'Ratatouille-Stuffed Bell Peppers with Herbed Quinoa',
 'cuisine_culture': 'French',
 'total_calories': 600,
 'ingredients': ['4 medium bell peppers (any color)',
  '1 medium zucchini, diced',
  '1 medium eggplant, diced',
  '1 medium onion, diced',
  '2 cloves garlic, minced',
  '1 medium tomato, diced',
  '1 teaspoon dried thyme',
  '1 teaspoon dried basil',
  '1 tablespoon olive oil',
  'Salt and pepper to taste',
  '1 cup quinoa, rinsed',
  '2 cups vegetable broth (low sodium)',
  '1 tablespoon fresh parsley, chopped',
  '1 tablespoon fresh basil, chopped',
  '1 tablespoon lemon juice'],
 'instructions': ['In a medium saucepan, combine the rinsed quinoa and vegetable broth. Bring to a boil over medium-high heat.',
  'Once boiling, reduce the heat to low, cover, and simmer for about 15 minutes or until the quinoa is fluffy and the liquid is absorbed.',
  'Remove from heat and let it sit covered for 5 minutes. Fluff with a fork and stir in parsley, basil, lemon juice, salt, 

## Get Meal Plan for Duration

In [75]:
days = int(duration)

In [76]:
full_meal_plan = {}

In [84]:
for i in range(days):
  daily_meal_result = daily_meals_chain.batch([
    {
        "calories_calculator_result": calories_calculator_result,
        "meal_type": "Breakfast",
        "user_input": user_input,
        "random_cuisine_hint": random.choice(cuisines)
    },
    {
        "calories_calculator_result": calories_calculator_result,
        "meal_type": "Lunch",
        "user_input": user_input,
        "random_cuisine_hint": random.choice(cuisines)
    },
    {
        "calories_calculator_result": calories_calculator_result,
        "meal_type": "Dinner",
        "user_input": user_input,
        "random_cuisine_hint": random.choice(cuisines)
    }
])
  daily_meal_agent_results = daily_meals_agent.batch([
    {"messages": [("user", json.dumps(daily_meal_result[0], indent=2))]},
    {"messages": [("user", json.dumps(daily_meal_result[1], indent=2))]},
    {"messages": [("user", json.dumps(daily_meal_result[2], indent=2))]}
])
  breakfast = daily_meal_agent_results[0]['messages'][1].content
  lunch = daily_meal_agent_results[1]['messages'][1].content
  dinner = daily_meal_agent_results[2]['messages'][1].content
  meal_plan_structure_result = meal_plan_structure_chain.batch([
    {
        "daily_meal_agent_results": breakfast
    },
    {
        "daily_meal_agent_results": lunch
    },
    {
        "daily_meal_agent_results": dinner
    }
])
  full_meal_plan[i] = {
      "breakfast": meal_plan_structure_result[0]["recipes"][0],
      "lunch": meal_plan_structure_result[1]["recipes"][0],
      "dinner": meal_plan_structure_result[2]["recipes"][0]
  }
  print(f"Day {i + 1} completed")
  print(full_meal_plan[i])

Day 1 completed
{'breakfast': {'name': 'Shakshuka with Spinach and Feta', 'cuisine_culture': 'Israeli', 'total_calories': 600, 'ingredients': ['2 tablespoons olive oil', '1 medium onion, diced', '2 cloves garlic, minced', '1 red bell pepper, diced', '1 teaspoon ground cumin', '1 teaspoon smoked paprika', '1 can (14 oz) diced tomatoes (no added sugar)', '4 large eggs', '2 cups fresh spinach', '1/2 cup feta cheese, crumbled', 'Salt and pepper to taste', 'Fresh parsley or cilantro for garnish (optional)'], 'instructions': ['In a large skillet, heat the olive oil over medium heat.', 'Add the diced onion and red bell pepper to the skillet. Sauté for about 5-7 minutes until the onion is translucent and the peppers are soft. Add the minced garlic and sauté for another minute until fragrant.', 'Stir in the ground cumin and smoked paprika, cooking for an additional minute to toast the spices.', 'Pour in the canned diced tomatoes (with their juices) and season with salt and pepper. Let the mixtu

In [85]:
full_meal_plan

{0: {'breakfast': {'name': 'Shakshuka with Spinach and Feta',
   'cuisine_culture': 'Israeli',
   'total_calories': 600,
   'ingredients': ['2 tablespoons olive oil',
    '1 medium onion, diced',
    '2 cloves garlic, minced',
    '1 red bell pepper, diced',
    '1 teaspoon ground cumin',
    '1 teaspoon smoked paprika',
    '1 can (14 oz) diced tomatoes (no added sugar)',
    '4 large eggs',
    '2 cups fresh spinach',
    '1/2 cup feta cheese, crumbled',
    'Salt and pepper to taste',
    'Fresh parsley or cilantro for garnish (optional)'],
   'instructions': ['In a large skillet, heat the olive oil over medium heat.',
    'Add the diced onion and red bell pepper to the skillet. Sauté for about 5-7 minutes until the onion is translucent and the peppers are soft. Add the minced garlic and sauté for another minute until fragrant.',
    'Stir in the ground cumin and smoked paprika, cooking for an additional minute to toast the spices.',
    'Pour in the canned diced tomatoes (with thei